In [1]:
from google.cloud import storage
import tensorflow as tf
from PIL import Image
import numpy as np

2023-10-16 00:18:43.660992: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-16 00:18:43.661118: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-16 00:18:43.672632: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


KeyboardInterrupt: 

In [2]:
Bucket_name = "tomato_disease_bucket"

Class_names = ["Tomato_Bacterial_spot", "Tomato_Early_blight", "Tomato_Late_blight","Tomato_Leaf_Mold",
                    "Tomato_Septoria_leaf_spot","Tomato__Target_Spot","Tomato__Tomato_YellowLeaf__Curl_Virus","Tomato__Tomato_mosaic_virus","Tomato_healthy"]

model = None

In [4]:
def download_blob(bucket_name, source_blob_name, destination, destination_file_name):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)

In [5]:
def predict(request):
    global model 
    if model is None:
        download_blob(
            Bucket_name,
            "Models/Disease_model.h5",
            "/tmp/Disease_model.h5"
        )

        model = tf.keras.models.load_model("/tmp/Disease_model.h5")
    
    image = request.files["file"]

    image = np.array(Image.open(image).convert("RGB"))#.resize((128,128))

    image_array = tf.expand_dims(image,0)


    predictions = model.predict(image_array)

    predictions = Class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0]),2))

    return{
            'class' : predictions,
            'confidence': float(confidence)
        }